In [1]:
#Read in Redlist

import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re
style.use('ggplot')

# Read in csv as dataframe
rl = em.read_csv_metadata("../finalRedlist.csv", encoding="ISO-8859-1", key='id')
# glance at first few rows

ar = em.read_csv_metadata("../trying.csv", encoding="ISO-8859-1", key='id')


c:\users\aparn\anaconda\envs\my_first_env\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [2]:
rl['block'] = 'yes'
ar['block'] = 'yes'

In [3]:

rl.rename(columns={'Unnamed: 0':'Unnamed'}, inplace=True)
ar.rename(columns={'Unnamed: 0':'Unnamed'}, inplace=True)
ar.columns

Index(['id', 'scientific_name', 'name', 'kingdom', 'phylum', 'class', 'order',
       'family', 'genus', 'size', 'threats', 'conservation', 'threat_keywords',
       'conservation_keywords', 'status', 'countries', 'country_count',
       'tCount', 'block'],
      dtype='object')

In [4]:
columns = ar.columns

#newColumns = []
#for column in columns:
    #new_column = column.replace(' ','')
    #ar=ar.rename(columns = {column:new_column})
#ar.to_csv('finalArkives.csv')

In [5]:
#Block on bogus attribute so that we can actually do
#blocking on genus
ob = em.OverlapBlocker()
C0 = ob.block_tables(rl, ar,'block', 'block', 
                     l_output_attrs=list(rl.columns.values), 
                     r_output_attrs=list(ar.columns.values))


0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:01:12


In [6]:
C1 = ob.block_candset(C0, 'genus', 'genus')


0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:02:30


In [7]:
#C1.head()

feat = em.get_features_for_matching(rl, ar)
feat


,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,name_name_jac_qgm_3_qgm_3,name,name,qgm_3,qgm_3,jaccard,<function name_name_jac_qgm_3_qgm_3 at 0x00000235DEFC3EA0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,name_name_cos_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,cosine,<function name_name_cos_dlm_dc0_dlm_dc0 at 0x00000235DFA29D08>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,name_name_jac_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,jaccard,<function name_name_jac_dlm_dc0_dlm_dc0 at 0x00000235DFA29F28>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,name_name_mel,name,name,None,None,monge_elkan,<function name_name_mel at 0x00000235DF327840>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,name_name_lev_dist,name,name,None,None,lev_dist,<function name_name_lev_dist at 0x00000235DF327400>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,name_name_lev_sim,name,name,None,None,lev_sim,<function name_name_lev_sim at 0x00000235DF327488>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,name_name_nmw,name,name,None,None,needleman_wunsch,<function name_name_nmw at 0x00000235DF327048>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,name_name_sw,name,name,None,None,smith_waterman,<function name_name_sw at 0x00000235DF327158>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,family_family_jac_qgm_3_qgm_3,family,family,qgm_3,qgm_3,jaccard,<function family_family_jac_qgm_3_qgm_3 at 0x00000235DF3270D0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,family_family_cos_dlm_dc0_dlm_dc0,family,family,dlm_dc0,dlm_dc0,cosine,<function family_family_cos_dlm_dc0_dlm_dc0 at 0x00000235DF3278C8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [18]:
H = em.extract_feature_vecs(C1, 
                            feature_table=feat, 
                            attrs_after='rtable_block',
                            show_progress=False) 

In [19]:

#C1 = C1.drop('rtable_block', 1)
#C1 = C1.drop('ltable_block', 1)
result = pd.concat([C1, H], axis=1)
#result = result.drop('rtable_block',1)


In [20]:
cols_to_drop = ['family_', 
                'scientific_name_', 
                'block', 
                'Unnamed',
                'name_name_cos_dlm_dc0_dlm_dc0',
                'name_name_mel', 
                'name_name_lev_dist',
                'name_name_lev_sim', 
                'name_name_nmw', 
                'name_name_sw'
               ]

for n in result.columns:
    for c in cols_to_drop:
        if c in n:
            result = result.drop(n,1)
result.columns

Index(['_id', 'ltable_id', 'rtable_id', 'ltable_name', 'ltable_genus',
       'ltable_family', 'ltable_ecology', 'ltable_countries',
       'ltable_threat_paragraph', 'ltable_conservation_paragraph',
       'ltable_pop_trend', 'ltable_status', 'ltable_country_count',
       'ltable_scientific_name', 'rtable_scientific_name', 'rtable_name',
       'rtable_kingdom', 'rtable_phylum', 'rtable_class', 'rtable_order',
       'rtable_family', 'rtable_genus', 'rtable_size', 'rtable_threats',
       'rtable_conservation', 'rtable_threat_keywords',
       'rtable_conservation_keywords', 'rtable_status', 'rtable_countries',
       'rtable_country_count', 'rtable_tCount', '_id', 'ltable_id',
       'rtable_id', 'name_name_jac_qgm_3_qgm_3',
       'name_name_jac_dlm_dc0_dlm_dc0', 'countries_countries_jac_qgm_3_qgm_3',
       'countries_countries_cos_dlm_dc0_dlm_dc0',
       'countries_countries_jac_dlm_dc0_dlm_dc0', 'countries_countries_mel',
       'countries_countries_lev_dist', 'countries_countr

In [11]:
label = em.split_train_test(C1, train_proportion=0.1039)
gold = label['train']
DS = label['test']


#Automatically label the training data
labels = []
matches = 0;
nonmatches = 0;
for index, row in gold.iterrows():
    if row['rtable_scientific_name'].strip().lower() == row['ltable_scientific_name'].strip().lower():
        labels.append(1)
        matches = matches + 1
    else:
        labels.append(0)
        nonmatches = nonmatches + 1
        
gold['label'] = labels
print('number of matches in training set:', matches)
print('number of nonmatches in training set:', nonmatches)

number of matches in training set: 28
number of nonmatches in training set: 239


In [12]:
gold


,_id,ltable_id,rtable_id,ltable_Unnamed,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,...,rtable_threats,rtable_conservation,rtable_threat_keywords,rtable_conservation_keywords,rtable_status,rtable_countries,rtable_country_count,rtable_tCount,rtable_block,label
2111715,2111715,6325,266,6325,dark red meranti,Shorea,Dipterocarpaceae,Terrestrial,Brunei Darussalam; Malaysia (Sarawak);,\r\r\n It is particularly threatened by logging operations because it grows too slowly to rea...,...,Dipterocarp forests have become amongst the most endangered in the world widely logged for use i...,Some subpopulations of this species are found in primary forest reserves where they receive vary...,,,Endangered,"['Tunisia', 'Algeria', 'Morocco']",3,0,yes,0
1167885,1167885,4380,147,4380,kerangas bubblenest frog,Philautus,Rhacophoridae,Terrestrial; Freshwater,Malaysia;,\r\r\n Loss of habitat due to the deforestation and fragmentation of peat swamps is the major...,...,The blue-eyed bush frog has lost a great amount of suitable habitat due to dam construction agri...,Some areas within the range of the blue-eyed bush frog are protected including the Sharavathi Va...,invasive;,protected;,Endangered,"['Cameroon', 'Gabon']",2,1,yes,0
2192219,2192219,7679,276,7679,ebners skink,Chalcides,Scincidae,Terrestrial,Morocco;,"\r\r\n This species has a restricted range, and is highly threatened by habitat loss principa...",...,Relatively little is known about the threats to Doumergue's skink. It is believed to be uncommon...,Doumergueâ??s skink occurs in the Chafarine Hunting Reserve in the Chafarinas islands Spain as w...,,protected;,Endangered,['China'],1,0,yes,0
5893603,5893603,4843,744,4843,"orangebellied racer, redbellied racer, saba racer",Alsophis,Dipsadidae,NaN,"Saint Kitts and Nevis; Bonaire, Sint Eustatius and Saba (Saba, Sint Eustatius);",\r\r\n Extirpated historically from Nevis and St Kitts due to the introduction of mongooses. ...,...,The red-bellied racer now occupies only 11 percent of its original range (5) largely due to the ...,Fortunately there are some systems in place to conserve the red-bellied racer. For example on St...,,protected;,Critically Endangered,"['Italy', 'Mexico']",2,0,yes,1
1369617,1369617,322,173,322,NaN,Mammillaria,Cactaceae,Terrestrial,Mexico (Sonora);,"\r\r\n The species' range is impacted by urban development, including tourism facilities, and...",...,Collectors have prized this distinctive and unusual cactus over the years and well-known sites h...,International trade in plants and seeds of wild origin is banned by the listing of <i>Mammillari...,,cites;,Endangered,"['Tunisia', 'Morocco', 'Algeria']",3,0,yes,0
7278065,7278065,4180,919,4180,NaN,Eleutherodactylus,Eleutherodactylidae,Terrestrial,Cuba;,\r\r\n The major threat to the species is disturbance of its habitat by tourists.\r\r\n\r\r\n...,...,Currently almost a quarter of all amphibian species on the British Virgin Islands are categorise...,A number of conservation recommendations have been made to protect the native fauna of the Briti...,,protected;,Critically Endangered,"['Dominican Republic', 'Haiti']",2,0,yes,0
8970546,8970546,2851,1133,2851,NaN,Encephalartos,Zamiaceae,Terrestrial,"Tanzania, United Republic of;","\r\r\n This species is affected by too frequent fires, which may affect regeneration. Changes...",...,Over the past few decades many South African cycads have become increasingly scarce in the wild ...,There are not known to be any specific conservation measures in place for this Critically Endang...,loss;,cites;protected;,Critically Endangered,['South Africa'],1,1,yes,0
1199440,1199440,4275,151,4275,"bernhards mantella, black mantella, tolongoina golden frog",Mantella,Mantellidae,Terrestrial; Freshwater,Madagascar;,\r\r\n The area where the species occurs is being degraded rapidly due to subsistence agricul...,...,Several thousand blue-legged mantellas are thought to be collected every year from some regions

In [13]:
em.to_csv_metadata(gold, './gold.csv')
em.to_csv_metadata(DS, './unlabelled.csv')

File already exists at ./gold.csv; Overwriting it
Metadata file already exists at ./gold.metadata. Overwriting it
File already exists at ./unlabelled.csv; Overwriting it
Metadata file already exists at ./unlabelled.metadata. Overwriting it


True

In [14]:
test_train = em.split_train_test(gold, train_proportion=0.7)
train = test_train['train']
test = test_train['test']
len(train)

186

In [15]:
em.to_csv_metadata(gold, './train.csv')
em.to_csv_metadata(DS, './test.csv')

File already exists at ./train.csv; Overwriting it
Metadata file already exists at ./train.metadata. Overwriting it
File already exists at ./test.csv; Overwriting it
Metadata file already exists at ./test.metadata. Overwriting it


True

In [21]:
em.to_csv_metadata(result, './candidate_set.csv')


File already exists at ./candidate_set.csv; Overwriting it
Metadata file already exists at ./candidate_set.metadata. Overwriting it


True

In [22]:
result

,_id,ltable_id,rtable_id,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,...,countries_countries_jac_dlm_dc0_dlm_dc0,countries_countries_mel,countries_countries_lev_dist,countries_countries_lev_sim,countries_countries_nmw,countries_countries_sw,country_count_country_count_exm,country_count_country_count_anm,country_count_country_count_lev_dist,country_count_country_count_lev_sim
25215,25215,1470,3,"adelaide pigmy bluetongue skink, pygmy bluetongue",Tiliqua,Scincidae,NaN,Australia;,NaN,NaN,...,0.0,0.511818,21,0.160000,-10.0,3.0,0,0.5,1.0,0.0
75598,75598,4363,9,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\r\r\n The major threat is drainage of wetland habitats and aquatic pollution of many waterwa...,"[""\n It is listed on Appendix III of the Bern Convention. 'Green frogs', including "", <em>R. ...",...,0.0,0.531032,21,0.160000,-2.0,10.0,1,1.0,0.0,1.0
75600,75600,4365,9,NaN,Pelophylax,Ranidae,Terrestrial; Freshwater,China;,\r\r\n The major threat is over-collecting for food; other threats include drought and water ...,['\n The Shapotou location is within a national nature reserve. There is a need to manage the...,...,0.0,0.455238,23,0.080000,-16.0,1.0,0,0.5,1.0,0.0
78778,78778,7543,9,karpathos frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Greece;,\r\r\n The major threat is habitat loss and degradation of suitable aquatic habitats. Althoug...,['\n It is listed on Appendix III of the Bern Convention. It probably occurs in a protected a...,...,0.0,0.470000,22,0.120000,-14.0,2.0,0,0.5,1.0,0.0
83513,83513,4363,10,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\r\r\n The major threat is drainage of wetland habitats and aquatic pollution of many waterwa...,"[""\n It is listed on Appendix III of the Bern Convention. 'Green frogs', including "", <em>R. ...",...,0.0,0.531032,21,0.160000,-2.0,10.0,1,1.0,0.0,1.0
83515,83515,4365,10,NaN,Pelophylax,Ranidae,Terrestrial; Freshwater,China;,\r\r\n The major threat is over-collecting for food; other threats include drought and water ...,['\n The Shapotou location is within a national nature reserve. There is a need to manage the...,...,0.0,0.455238,23,0.080000,-16.0,1.0,0,0.5,1.0,0.0
86693,86693,7543,10,karpathos frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Greece;,\r\r\n The major threat is habitat loss and degradation of suitable aquatic habitats. Althoug...,['\n It is listed on Appendix III of the Bern Convention. It probably occurs in a protected a...,...,0.0,0.470000,22,0.120000,-14.0,2.0,0,0.5,1.0,0.0
105744,105744,2849,13,"alexandria cycad, dune cycad",Encephalartos,Zamiaceae,Terrestrial,South Africa (Eastern Cape Province);,\r\r\n The major threat to this species is removal by collectors. This has been the main caus...,['\n This species is listed on Appendix I of the CITES Appendices. Populations occur within t...,...,0.0,0.470760,35,0.078947,-26.0,2.0,1,1.0,0.0,1.0
105745,105745,2850,13,chimanimani cycad,Encephalartos,Zamiaceae,Terrestrial,Mozambique;,\r\r\n This species is threatened due to over-collecting for ornamental purposes.\r\r\n\r\r\n...,['\n This species is listed on Appendix I of the CITES Appendices.\n\n \n '],...,0.0,0.398148,11,0.083333,-2.0,1.0,1,1.0,0.0,1.0
105746,105746,2851,13,NaN,Encephalartos,Zamiaceae,Terrestrial,"Tanzania, United Republic of;","\r\r\n This species is affected by too frequent fires, which may affect regeneration. Changes...",['\n This species is listed on Appendix I of the CITES Appendices. This species possibly occu...,...,0.0,0.366667,27,0.100000,-18.0,2.0,1,1.0,0.0,1.0
